# Starup.ML - Global Terrorism Attact Attribution Challenge
## Chris Cronin

This github post is part of a submission for Startup.ML's San Francisco fellowship. To showcase my knowledge and skill level with data science, I chose to work on the Global Terrorism Attack Attribution problem, which asks to build a model that can predict what terrorist group is responsible for attacks where the terrorist group is unknown. The decision tree grid search classification model had the highest accuracy (95%) for predicting the terrorist group of in sample test data.

The data for this challenge can be found at https://www.start.umd.edu/gtd/

Download this file: globalterrorismdb_0615dist

Save it as a csv with the name: terrorist_data.csv

## Agenda

1. Problem Statement
2. Data Extraction / Loading
3. Data Cleaning (Fill Null values)
4. Model Setup
5. Model Evaluation
6. Insights / Conclusion

## Imports

In [1]:
%matplotlib inline
import os
import json
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from seaborn import plt
import random 
from __future__ import division
from matplotlib import pyplot
import matplotlib.pyplot as plt 
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
pd.set_option('max_info_columns', 200)

/Users/Cronin/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Part 1: Problem Statement

- Global Terrorism Database (GTD) is an open-source database including information on terrorist events around the world from 1970 through 2014. Some portion of the attacks have not been attributed to a particular terrorist group.
- Use attack type, weapons used, description of the attack, etc. to build a model that can predict what group may have been responsible for an incident. 

## Part 2: Data Extraction / Loading
 - globalterrrorismdb_0615dist.xlsx can be downloaded from [Global Terrorism Database](/http://www.start.umd.edu/gtd/)
 - Converted to csv file called "terrorist_data.csv"
 - Review GTD Codebook with feature descriptions

In [3]:
terrorism = pd.read_csv('terrorist_data.csv')

/Users/Cronin/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (4,6,13,31,33,53,61,62,63,75,78,91,93,95,113,114,120) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


- Filtered dataset to include only information starting from 1998 because many of the GTD Codebook Features only apply to terrorism incidents occuring after 1997. 

In [4]:
terrorism_98 = terrorism[terrorism.iyear >= 1998]

- Binary Features: Replace Columns that contain "9" or "99" for unkown values with np.nan

In [5]:
def fill_nines(df):
    missing_value_as_neg_nines = ["ishostkid", "property","INT_LOG","INT_IDEO","INT_MISC","INT_ANY"]
    for col in missing_value_as_neg_nines:
        df.loc[df[col] == -9, col] = np.nan
    missing_value_as_neg_ninetynines = ["nhostkid","nhostkidus","nhours","nperpcap","nperps"]
    for col in missing_value_as_neg_ninetynines:
        df.loc[df[col] == -99, col] = np.nan
    return df

In [6]:
terrorism_98 = fill_nines(terrorism_98)

/Users/Cronin/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


- "gname" response feature colum contains the name of the terrorist group that carried out the attack.
- Filtered dataframe to create out of sample data where "gname" for rows is "Unknown"
- Filtered datafame for rows with a "gname" for terrorist group names that occur at least 50 times in the dataset

In [7]:
# create out of sample dataframe where gname is unknown or blank
terrorism_98_nognames = terrorism_98[terrorism_98.gname == "Unknown"]

# Change existing terrorism_98 dataframe to only include rows that include a gname
terrorism_98 = terrorism_98[terrorism_98.gname != "Unknown"]

# Filter for gname responses that at least occur 50 times 
gname_counts = terrorism_98.gname.value_counts()
gnames_over_threshold = gname_counts[gname_counts > 50].index
terrorism_98 = terrorism_98[terrorism_98.gname.isin(gnames_over_threshold)]

In [55]:
print("number of unique gnames:",terrorism_98.gname.unique().size)
print("number of unique gnames:",terrorism_98_nognames.gname.unique().size) # only one: "unkown"

('number of unique gnames:', 75)
('number of unique gnames:', 1)


- Function to filter dataframe by quantity of null values
- Threshold: 80% of rows filled or 20% of rows not filled

In [9]:
def get_feature_subset_nulls(df, response_col, threshold=0.2):
    """
    Returns Features that have less than 20% missing values.
    """
    #Applying per column: Defined by axis=0
    feature_null_counts = df.apply(lambda col: col.isnull().sum(), axis=0)

    num_rows = len(df)
    features = feature_null_counts[feature_null_counts < (threshold * num_rows)].index.difference([response_col])
    # Exclude features that are Text (these end with _txt)
    features = features[~features.str.endswith("_txt")]

    return features

In [10]:
features = get_feature_subset_nulls(terrorism_98, "gname")
features

Index([u'INT_ANY', u'INT_MISC', u'attacktype1', u'city', u'claimed', u'corp1',
       u'country', u'crit1', u'crit2', u'crit3', u'dbsource', u'doubtterr',
       u'eventid', u'extended', u'guncertain1', u'iday', u'imonth',
       u'ishostkid', u'iyear', u'latitude', u'longitude', u'multiple',
       u'natlty1', u'nkill', u'nkillter', u'nkillus', u'nperpcap', u'nwound',
       u'nwoundte', u'nwoundus', u'property', u'provstate', u'region',
       u'scite1', u'specificity', u'success', u'suicide', u'summary',
       u'target1', u'targsubtype1', u'targtype1', u'vicinity', u'weapsubtype1',
       u'weaptype1'],
      dtype='object')

- numeric_features below includes all non object/text features from "features" above except eventid, which is unnecessary in the model because it represents a unique id per attack.

In [11]:
numeric_features = ['INT_ANY','INT_IDEO','INT_LOG','INT_MISC', 
                    'attacktype1','claimed', 'country', 'crit1', 
                    'crit2', 'crit3', 'doubtterr','extended', 
                    'guncertain1','iday','imonth','ishostkid', 
                    'iyear','multiple','natlty1', 'nkill', 
                    'nkillter', 'nkillus', 'nperpcap','nperps', 
                    'nwound', 'nwoundte', 'nwoundus', 'property', 
                    'region', 'specificity', 'success', 'suicide', 
                    'targsubtype1', 'targtype1', 'vicinity', 
                    'weapsubtype1', 'weaptype1']

- text_featues includes all object/text columns from "features" above as well as two new features: "location" and "target2", which I felt could be important features that didn't meet the threshold of 0.2

In [12]:
text_features = ['provstate','location','city',
                 'summary','target1','target2',
                 'scite1','dbsource','corp1']

all_features = numeric_features + text_features

## Part 3: Data Cleaning (Fill Null Values)

- Numeric Binary Features: Fill null values with probability weighted 0's and 1's from existing probability distribution of filled values

In [13]:
def binary_fill(df, fill_values):
    binary_columns_notfull = ["ishostkid", "property","INT_LOG","INT_IDEO","INT_MISC","INT_ANY",'guncertain1']
    for col in binary_columns_notfull:
        if col not in fill_values:
            col_pd = df[col].value_counts()/df[col].value_counts().sum()
            fill_values[col] = col_pd[0]
        df.loc[df[col].isnull(), col] = np.random.choice([0,1], size=df[col].isnull().sum(), p=[fill_values[col], 1 - fill_values[col]])
    return df

- Numeric Range Features: Replace null values with random choice of range integers weighted by their existing probability distribution of filled values

In [14]:
def range_fill(df, fill_values):
    range_columns_notfull = ["targsubtype1","weapsubtype1","natlty1","specificity"]
    for col in range_columns_notfull:
        if col not in fill_values:
            col_dist = df[col].value_counts()/df[col].value_counts().sum()
            fill_values[col] = col_dist
        
        col_val_missing = df[col].isnull()
        df.ix[col_val_missing, col] = np.random.choice(fill_values[col].index, size=col_val_missing.sum(), p=fill_values[col].values)
    return df

- Numeric Value Features: Replace null values with median value

In [15]:
def numeric_fill(df, fill_values):
    numeric_columns_notfull = ["nkill","nkillus","nkillter","nwound","nwoundus","nwoundte",'nperpcap','nperps']
    for col in numeric_columns_notfull:
        if col not in fill_values:
            fill_values[col] = df[col].median()

        df.ix[df[col].isnull(), col] = fill_values[col]
    return df

- Function that fills numeric feature null values based on three functions above: binary_fill,range_fill, and numeric_fill

In [16]:
def gen_numeric_features(features):
    
    binary_fill_values = {}
    range_fill_values = {}
    numerc_fill_values = {}
    
    def transform_numeric_features(df):
        df = df[features]
        df = binary_fill(df, binary_fill_values)
        df = range_fill(df, range_fill_values)
        df = numeric_fill(df, numerc_fill_values)
        
        return df

    return transform_numeric_features

## Part 4: Model Setup

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

### Combining feature extraction steps
- Description here from Kevin Markham's Machine Learning with Text in Python Part-Time Course
- **`FeatureUnion`** applies a list of transformers in parallel to the input data (not sequentially), then **concatenates the results**. This is useful for combining several feature extraction mechanisms into a single transformer.
- A pipeline sequentially applies a list of transformers and a final estimator. 
- Function below creates a union through a pipeline of both numeric and text features transformed to a document-term matrix

[make_union documentation](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_union.html) and
[Pipeline documentation](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) and [FeatureUnion documentation](http://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html)

In [19]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import FunctionTransformer

pipelines = []

def get_feature_text(col):
    
    def get_text(df):
        return df[col].fillna("")
    
    return get_text

# Build a Document Term Matrix using CountVectorizer for ALL Text Columns defined in text_features list.
for col in text_features:
    get_text_ft = FunctionTransformer(get_feature_text(col), validate=False)
    pipelines.append(make_pipeline(get_text_ft, CountVectorizer(decode_error='ignore')))

# Add the Numeric Data into the pipeline
get_numeric_features = gen_numeric_features(numeric_features)
pipelines.append(FunctionTransformer(get_numeric_features, validate=False))

union = make_union(*pipelines)

- Split the dataset into train/test/split partitions

In [21]:
X_train, X_test, y_train, y_test = train_test_split(terrorism_98[all_features], terrorism_98.gname, 
                                                    test_size=0.3, random_state=15)

- Create new X_train, X_test based on the combined union of numeric and text features transformed to a document-type matrix

In [22]:
X_train_dtm = union.fit_transform(X_train)
X_test_dtm = union.transform(X_test)

- Create new X set to represent out of sample data that models will use to predict terrorist attack group
- Note, because this is out of sample, accuracy can't be determined unless the Global Terrorism Database can verify

In [23]:
x_new_dtm = union.transform(terrorism_98_nognames)

## Part 5: Model Evaluation

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

### Logistic Regression

In [24]:
params_c = np.logspace(-3, 3, 7) # 10^-3 to 10^3

In [25]:
from sklearn.grid_search import GridSearchCV

In [26]:
logreg_grid_search = GridSearchCV(LogisticRegression(), param_grid={'C': np.logspace(-3, 3, 7)})
logreg_grid_search.fit(X_train_dtm, y_train)
log_pred_search_pred = logreg_grid_search.predict(X_test_dtm)
log_pred_search_pred

array(['Taliban', 'Garo National Liberation Army', 'Boko Haram', ...,
       'Taliban', 'Baloch Republican Army (BRA)', 'Chechen Rebels'], dtype=object)

In [27]:
print metrics.accuracy_score(y_test, log_pred_search_pred)
print metrics.classification_report(y_test, log_pred_search_pred)
# Precision: True Pos / (True pos + False Positives)
# Recall: True Pos / (true pos + false negatives)

0.9261829653
             precision    recall  f1-score   support

Abu Sayyaf Group (ASG)       0.98      0.98      0.98        87
Al-Aqsa Martyrs Brigade       0.88      0.88      0.88        43
Al-Nusrah Front       0.94      0.30      0.45        50
  Al-Qa`ida       1.00      0.38      0.56        13
Al-Qa`ida in Iraq       0.81      0.99      0.89       190
Al-Qa`ida in the Arabian Peninsula (AQAP)       0.97      0.99      0.98       229
Al-Qa`ida in the Lands of the Islamic Maghreb (AQLIM)       0.74      0.74      0.74        68
 Al-Shabaab       0.97      1.00      0.98       513
Algerian Islamic Extremists       1.00      1.00      1.00       106
Allied Democratic Forces (ADF)       0.96      0.88      0.92        25
Animal Liberation Front (ALF)       1.00      0.85      0.92        20
Ansar Bayt al-Maqdis (Ansar Jerusalem)       1.00      1.00      1.00        26
Armed Islamic Group (GIA)       0.82      0.89      0.85        36
Baloch Liberation Army (BLA)       0.97      

In [29]:
logreg_grid_search.best_params_

{'C': 1000.0}

In [30]:
logreg_grid_search.grid_scores_

[mean: 0.69746, std: 0.00418, params: {'C': 0.001},
 mean: 0.88551, std: 0.00936, params: {'C': 0.01},
 mean: 0.93294, std: 0.01248, params: {'C': 0.10000000000000001},
 mean: 0.92872, std: 0.01120, params: {'C': 1.0},
 mean: 0.93186, std: 0.01243, params: {'C': 10.0},
 mean: 0.93294, std: 0.01370, params: {'C': 100.0},
 mean: 0.93375, std: 0.00960, params: {'C': 1000.0}]

In [31]:
logreg_grid_search.best_score_

0.93374797187669012

In [32]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [33]:
confusion_matrix(y_test,logreg_grid_search.predict(X_test_dtm))
# diagnoal is good. Anything not in diagnoal is misclassified

array([[85,  0,  0, ...,  0,  0,  0],
       [ 0, 38,  0, ...,  0,  0,  0],
       [ 0,  0, 15, ...,  0,  0,  0],
       ..., 
       [ 0,  0,  0, ..., 17,  0,  0],
       [ 0,  0,  0, ...,  0, 78,  0],
       [ 0,  0,  0, ...,  0,  0,  7]])

In [34]:
print classification_report(y_test,logreg_grid_search.predict(X_test_dtm))

             precision    recall  f1-score   support

Abu Sayyaf Group (ASG)       0.98      0.98      0.98        87
Al-Aqsa Martyrs Brigade       0.88      0.88      0.88        43
Al-Nusrah Front       0.94      0.30      0.45        50
  Al-Qa`ida       1.00      0.38      0.56        13
Al-Qa`ida in Iraq       0.81      0.99      0.89       190
Al-Qa`ida in the Arabian Peninsula (AQAP)       0.97      0.99      0.98       229
Al-Qa`ida in the Lands of the Islamic Maghreb (AQLIM)       0.74      0.74      0.74        68
 Al-Shabaab       0.97      1.00      0.98       513
Algerian Islamic Extremists       1.00      1.00      1.00       106
Allied Democratic Forces (ADF)       0.96      0.88      0.92        25
Animal Liberation Front (ALF)       1.00      0.85      0.92        20
Ansar Bayt al-Maqdis (Ansar Jerusalem)       1.00      1.00      1.00        26
Armed Islamic Group (GIA)       0.82      0.89      0.85        36
Baloch Liberation Army (BLA)       0.97      0.83      0.9

### KNN

In [35]:
knn_grid_search = GridSearchCV(KNeighborsClassifier(), param_grid={'n_neighbors': range(3, 22, 2)})

In [36]:
knn_grid_search.fit(X_train_dtm, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [3, 5, 7, 9, 11, 13, 15, 17, 19, 21]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [37]:
X_train_dtm.shape

(18490, 75079)

In [38]:
knn_grid_search.best_score_

0.71411573823688479

In [39]:
knn_grid_search.best_params_

{'n_neighbors': 5}

In [40]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_dtm, y_train)
knn_pred = knn.predict(X_test_dtm)
print metrics.accuracy_score(y_test, knn_pred)

0.721892744479


### Naive Bayes

In [54]:
nb_model = GaussianNB()
nb_model.fit(X_train_dtm.toarray(),y_train) # Use X.toarray() to convert to a dense numpy array due to sparse matrix
nb_model.score(X_test_dtm.toarray(),y_test)

0.83848580441640375

### Decision Tree

In [42]:
params_c = np.logspace(-3, 3, 7) # 10^-3 to 10^3
logreg_grid_search = GridSearchCV(LogisticRegression(), param_grid={'max_depth': [100,500,1000,10000]})
# C is cost function. The lower value of C, the lower the complexity of the model. 

In [43]:
tree_grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid={'max_depth': np.logspace(-3, 3, 6)})
tree_grid_search.fit(X_train_dtm, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_depth': array([  1.00000e-03,   1.58489e-02,   2.51189e-01,   3.98107e+00,
         6.30957e+01,   1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [44]:
tree_grid_search.best_params_

{'max_depth': 1000.0}

In [45]:
tree_grid_search.best_score_

0.94786371011357495

### Random Forest

In [46]:
forest_grid_search = GridSearchCV(RandomForestClassifier(), param_grid={'n_estimators': [10,100,1000]})
# of trees in n_estimators

In [47]:
forest_grid_search.fit(X_train_dtm, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [10, 100, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [48]:
forest_grid_search.best_params_

{'n_estimators': 1000}

In [49]:
forest_grid_search.best_score_

0.94726879394267172

### SVC

In [50]:
svc_grid_search = GridSearchCV(SVC(), param_grid={'C': [.01,1,100]})

In [51]:
svc_grid_search.fit(X_train_dtm, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=1, param_grid={'C': [0.01, 1, 100]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [52]:
svc_grid_search.best_params_

{'C': 100}

In [53]:
svc_grid_search.best_score_

0.88561384532179555

### Best Model

In [2]:
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import VotingClassifier

In [ ]:
clf1 = LogisticRegression(C=1000,random_state=1)
clf2 = RandomForestClassifier(n_estimators=1000,random_state=1)
clf3 = KNeighborsClassifier(n_neighbors=5)
clf4 = SVC(C=100)
# Hard: Everyone has a vote and will take a majority of the "Yes" votes. 
# Soft: It adds all the probabilites, then averages the probabilites, takes the best.

In [62]:
eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('knn', clf3),('SVC',clf4)], voting='hard')
eclf1 = eclf1.fit(X_train_dtm, y_train)
print(eclf1.predict(X_test_dtm))
print(eclf1.score(X_test_dtm,y_test)) #automatically does cross validation

['Taliban' 'Garo National Liberation Army' 'Boko Haram' ..., 'Taliban'
 'Baloch Republican Army (BRA)' 'Chechen Rebels']
0.932870662461


In [64]:
eclf2 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('knn', clf3)],voting='soft')
eclf2 = eclf2.fit(X_train_dtm, y_train)
print(eclf2.predict(X_test_dtm))
print(eclf2.score(X_test_dtm,y_test))

['Taliban' 'Garo National Liberation Army' 'Boko Haram' ..., 'Taliban'
 'Baloch Republican Army (BRA)' 'Chechen Rebels']
0.926561514196


## Part 6: Insights/Conclusions

- Created stored predictions for out of sample data based on each model

In [ ]:
y_new_logreg = logreg_grid_search.predict(x_new_dtm,delegate='estimator')

In [68]:
y_new_knn = knn.predict(x_new_dtm)

In [ ]:
y_new_nb = nb_model.predict(x_new_dtm.toarray())

In [ ]:
y_new_tree = tree_grid_search.predict(x_new_dtm,delegate='estimator')

In [ ]:
y_new_forest = forest_grid_search.predict(x_new_dtm,delegate='estimator')

In [ ]:
y_new_svc = svc_grid_search.predict(x_new_dtm,delegate='estimator')

#### Model Ranking by Accuracy Score

- Tree Grid Search: 0.9478
- Forest Grid Search: 0.9472
- Logistic Regression Grid Search: 0.9337 
- eclf1 Voting Classifier: Hard Vote: 0.9328
- eclf2 Voting Classifier: Soft Vote: 0.9265
- SVC Grid Search: 0.8856
- Naive Bayes: 0.8384
- KNN Grid Search: 0.7218

#### These machine learning models classified the terrorist attack group with fairly high accuracy. 
#### The decision tree grid seach model perfomed the best with an accuracy score of .9478.

Limitations: It is important to note that these models were tested on a dataset with data starting from the year 1998 and where terrorist attack groups were filtered to include only attacks by a group that occured at least 50 times in the dataset.  